In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time, os

# ─── CONFIG ─────────────────────────────────────────────────────────────
URL               = "https://wbtenders.gov.in/nicgep/app?page=WebTenderStatusLists&service=page"
DEPARTMENT_VALUE  = "257"   # ← set this to the Assam-specific value
DEPARTMENT_NAME   = "TAMRALIPTA CO-OPERATIVE SPINNING MILLS LTD."      # will be populated at runtime
FROM_DATE         = "01/01/2021"
TO_DATE           = "31/12/2024"
OUTPUT_XLS        = "TAMRALIPTA CO-OPERATIVE SPINNING MILLS LTD..xlsx"

# ─── BROWSER SETUP ──────────────────────────────────────────────────────
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(URL)
time.sleep(2)

# ─── PICK DEPARTMENT ────────────────────────────────────────────────────
select = Select(driver.find_element(By.ID, "OrganName"))
select.select_by_value(DEPARTMENT_VALUE)
DEPARTMENT_NAME = select.first_selected_option.text.strip()
print(f"🔎 Using Department: {DEPARTMENT_NAME}")

# ─── SET DATE FILTERS ───────────────────────────────────────────────────
for fld_id, val in [("publishedFromDate", FROM_DATE), ("publishedToDate", TO_DATE)]:
    el = driver.find_element(By.ID, fld_id)
    driver.execute_script("arguments[0].removeAttribute('readonly')", el)
    el.clear()
    el.send_keys(val)

# ─── CAPTCHA ───────────────────────────────────────────────────────────
png = "captcha_kolkata.png"
with open(png, "wb") as f:
    f.write(driver.find_element(By.ID, "captchaImage").screenshot_as_png)
print(f"✔ CAPTCHA saved to {png}. Please open it to read.")
captcha = input("Enter CAPTCHA: ")
driver.find_element(By.ID, "captchaText").send_keys(captcha)
driver.find_element(By.ID, "Search").click()
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "tabList")))

# ─── FIELD EXTRACTOR ────────────────────────────────────────────────────
def get_field_value(label):
    target = label.strip().lower()
    captions = driver.find_elements(By.XPATH, "//td[contains(@class,'caption')]")
    for cap in captions:
        if target in cap.text.strip().lower():
            try:
                return cap.find_element(By.XPATH, "following-sibling::td[1]").text.strip()
            except:
                return ""
    return ""

# ─── SCRAPE & PAGINATE ──────────────────────────────────────────────────
all_data = []

while True:
    rows = driver.find_elements(By.XPATH, "//table[@id='tabList']//tr[@class='even' or @class='odd']")
    for idx in range(len(rows)):
        try:
            # refresh row list each iteration
            rows = driver.find_elements(By.XPATH, "//table[@id='tabList']//tr[@class='even' or @class='odd']")
            row = rows[idx]
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) < 6:
                continue

            sno, tid, title, orgs, stage, status = [c.text.strip() for c in cells[:6]]

            # open detail popup
            row.find_element(By.XPATH, ".//td[last()]/a").click()
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "DirectLink"))).click()
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            driver.switch_to.window(driver.window_handles[-1])
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))

            # extract fields
            record = {
                "Department":               DEPARTMENT_NAME,
                "S.No":                     sno.rstrip('.'),
                "Tender ID":                tid,
                "Title & Ref No.":          title,
                "Organisation Chain":       orgs,
                "Tender Stage":             stage,
                "Status":                   status,
                "Publish Date":             get_field_value("Publish Date"),
                "Bid Submission End":       get_field_value("Bid Submission End Date"),
                "Tender Value (₹)":         get_field_value("Tender Value"),
                "Location":                 get_field_value("Location"),
                "Pincode":                  get_field_value("Pincode"),
                "Period Of Work (Days)":    get_field_value("Period Of Work"),
                "Address":                  get_field_value("Address"),
            }
            all_data.append(record)
            print(f"✔ Row {sno}: {tid} extracted")

            # close detail and return to list
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(text(),'Back')]"))
            ).click()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "tabList")))

        except Exception as e:
            print(f"❌ Row {idx+1} error: {e}")
            driver.switch_to.window(driver.window_handles[0])
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "tabList")))

    # move to next page, if any
    try:
        driver.find_element(By.ID, "loadNext").click()
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "tabList")))
    except:
        break

# ─── EXPORT TO EXCEL ───────────────────────────────────────────────────
df = pd.DataFrame(all_data)
df.to_excel(OUTPUT_XLS, index=False)
print(f"\n✅ Done! Saved {len(all_data)} records to {OUTPUT_XLS}")

driver.quit()


🔎 Using Department: TAMRALIPTA CO-OPERATIVE SPINNING MILLS LTD.
✔ CAPTCHA saved to captcha_kolkata.png. Please open it to read.


Enter CAPTCHA:  yD3384


✔ Row 1.: 2024_TCSML_789072_1 extracted
✔ Row 2.: 2024_TCSML_783850_1 extracted
✔ Row 3.: 2024_TCSML_783779_1 extracted
✔ Row 4.: 2024_TCSML_763173_1 extracted
✔ Row 5.: 2024_TCSML_760228_1 extracted
✔ Row 6.: 2024_TCSML_751858_1 extracted
✔ Row 7.: 2024_TCSML_751838_1 extracted
✔ Row 8.: 2024_TCSML_711198_1 extracted
✔ Row 9.: 2024_TCSML_711157_1 extracted
✔ Row 10.: 2024_TCSML_707621_1 extracted
✔ Row 11.: 2024_TCSML_695806_1 extracted
✔ Row 12.: 2024_TCSML_691862_1 extracted
✔ Row 13.: 2024_TCSML_687648_1 extracted
✔ Row 14.: 2024_TCSML_677389_1 extracted
✔ Row 15.: 2024_TCSML_677000_1 extracted
✔ Row 16.: 2024_TCSML_644771_1 extracted
✔ Row 17.: 2023_TCSML_620822_1 extracted
✔ Row 18.: 2023_TCSML_610287_1 extracted
✔ Row 19.: 2023_TCSML_601934_1 extracted
✔ Row 20.: 2023_TCSML_601272_1 extracted
✔ Row 21.: 2023_TCSML_600872_1 extracted
✔ Row 22.: 2023_TCSML_600046_1 extracted
✔ Row 23.: 2023_TCSML_591664_1 extracted
✔ Row 24.: 2023_TCSML_591444_1 extracted
✔ Row 25.: 2023_TCSML_591